# Input parameters 

In [ ]:
from glob import glob
exclude = []
fls = glob('../../../2_simulations/1_MHETase_5MHET_nowat_dt100ps/*dt100ps.xtc')
fls.sort()
traj_list = [v for i,v in enumerate(fls) if i not in exclude]
len(traj_list)
pdb_file = '../../../2_simulations/1_MHETase_5MHET_nowat_dt100ps/01_MHETase_5MHET-md_nowat.pdb'

In [ ]:
pwd

In [4]:
figure_name = 'spm'

#### Specify the algorithm for finding community structures: 'fastgreedy', 'edge_betweness', 'walktrap'

In [38]:
#community_method = 'edge_betweness'
community_method = 'fastgreedy'

### Run the code below

In [6]:
import numpy as np 
import mdtraj 
import pandas as pd 
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt 

In [7]:
import sys
import string
import re
import math
import csv
import igraph
import random
import cairo

In [8]:
import pyemma
import pyemma.coordinates as coor
import pyemma.msm as msm
import pyemma.plots as mplt
pyemma.__version__

'2.5.9'

In [9]:
ref = mdtraj.load(pdb_file)
top = ref.topology
#ref = ref.atom_slice(top.select('protein and not (resname CAL)'))

In [10]:
def get_protein_res_num(top, chain_id=0):
    return len(top.select(f'chainid {chain_id} and protein and name CA'))

def rmsd_of_residues(trajectory, reference=None, chain_id=0):
    cv = []
    top = trajectory.topology
    
    # Align to reference
    selection1 =  top.select(f'chainid {chain_id} and protein and backbone')
    trajectory.superpose(reference, atom_indices=selection1, ref_atom_indices=selection1)
    
    for i in top.residues:
        #print(i)
        if i.is_protein and i.chain.index == chain_id and i.name != 'CAL':

            # Compute the RMSD manually without additional alignment       
            atom_indexs = top.select(f'resid {i.index}')
            selection2 = np.array([a for a in atom_indexs if top.atom(a).element != mdtraj.core.element.hydrogen])
            rmsd_of_res = np.sqrt(3*np.mean((trajectory.xyz[:, selection2, :] - reference.xyz[:, selection2, :])**2, axis=(1,2)))
            cv.append(rmsd_of_res)
    cv = np.array(cv)
    return cv.transpose()

In [11]:
traj_list[:2]

['../../../2_simulations/1_MHETase_5MHET_nowat_dt100ps/01_MHETase_5MHET-md_nowat_dt100ps.xtc',
 '../../../2_simulations/1_MHETase_5MHET_nowat_dt100ps/02_MHETase_5MHET-md_nowat_dt100ps.xtc']

In [20]:
try:
    df = pd.read_hdf('feat_data_df.h5', key='feat_data_df')
except:
    feat = pyemma.coordinates.featurizer(pdb_file)
    feat_dim = get_protein_res_num(top, chain_id=0)
    feat.add_custom_func(rmsd_of_residues, dim=feat_dim, reference=ref, chain_id=0)
    feat_data = pyemma.coordinates.load(traj_list, feat)
    df = pd.DataFrame(np.vstack(feat_data))
    df.to_hdf('feat_data_df.h5', key='feat_data_df')

In [21]:
df.shape

(420070, 558)

In [56]:
df

,0,1,2,3,4,5,6,7,8,9,...,548,549,550,551,552,553,554,555,556,557
0,0.406822,0.331927,0.132408,0.136146,0.069360,0.284398,0.114326,0.086559,0.095129,0.284427,...,1.574149,0.812675,0.400428,0.249611,0.163877,0.144600,0.175688,0.207325,0.157347,0.188786
1,0.403427,0.358114,0.180834,0.112714,0.144636,0.329119,0.150981,0.149064,0.174646,0.233489,...,1.571714,0.892976,0.530524,0.223342,0.131579,0.109379,0.117679,0.135033,0.171020,0.208706
2,0.390430,0.344669,0.136494,0.108854,0.077369,0.293445,0.062346,0.078270,0.097199,0.196360,...,1.399674,0.835839,0.390853,0.202077,0.104471,0.071345,0.121425,0.179160,0.261982,0.310902
3,0.323098,0.290010,0.143686,0.099850,0.119723,0.244140,0.135610,0.117300,0.092572,0.160583,...,1.456726,0.942012,0.427168,0.228941,0.109315,0.119934,0.092454,0.086333,0.072859,0.138443
4,0.336233,0.265388,0.107137,0.098466,0.097223,0.304615,0.134356,0.126282,0.135998,0.178912,...,1.492016,0.927405,0.486415,0.239271,0.143040,0.116669,0.123571,0.121411,0.085896,0.191772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420065,0.449197,0.342288,0.073494,0.077829,0.082730,0.084053,0.132038,0.052124,0.083967,0.210871,...,0.134143,0.122419,0.129342,0.101124,0.132988,0.100169,0.090993,0.070202,0.083486,0.156455
420066,0.473219,0.380945,0.135357,0.150951,0.095056,0.213402,0.153263,0.086801,0.121574,0.230870,...,0.168424,0.129304,0.097083,0.118077,0.153099,0.120260,0.145370,0.112814,0.062291,0.178836
420067,0.496958,0.370690,0.106837,0.095558,0.132651,0.139355,0.156094,0.080009,0.083103,0.191322,...,0.165951,0.154877,0.160373,0.117417,0.114897,0.095244,0.124395,0.207809,0.268157,0.216101
420068,0.528062,0.410621,0.128515,0.093053,0.079636,0.165928,0.155137,0.106562,0.159645,0.278144,...,0.192293,0.185060,0.166005,0.165025,0.189883,0.174878,0.194906,0.195272,0.164273,0.177692


In [22]:
try:
    corr = pd.read_hdf('feat_data_df.h5',  key='corr')
except:
    corr = df.corr()
    corr.to_hdf('feat_data_df.h5', key='corr')

In [23]:
corr

,0,1,2,3,4,5,6,7,8,9,...,548,549,550,551,552,553,554,555,556,557
0,1.000000,0.898202,0.667760,0.395738,0.032168,-0.029886,-0.137668,-0.180642,-0.169305,0.070113,...,-0.105884,-0.112967,-0.129960,-0.120251,-0.081035,-0.045691,-0.037065,-0.027702,-0.006768,-0.024538
1,0.898202,1.000000,0.703401,0.466338,0.129166,0.017560,-0.082936,-0.099542,-0.067787,0.077940,...,-0.064703,-0.066575,-0.088224,-0.084698,-0.055016,-0.012598,-0.017195,-0.021747,-0.004366,-0.021791
2,0.667760,0.703401,1.000000,0.785567,0.528038,0.200217,0.254961,0.258384,0.234355,0.086110,...,-0.045900,-0.027580,0.000533,0.020102,0.011743,0.025667,0.002583,-0.016028,-0.020960,-0.027032
3,0.395738,0.466338,0.785567,1.000000,0.701916,0.324589,0.357932,0.337834,0.310798,0.141749,...,-0.036540,-0.020913,0.021856,0.042189,0.051028,0.052823,0.029776,-0.021659,-0.044240,-0.048868
4,0.032168,0.129166,0.528038,0.701916,1.000000,0.409093,0.764842,0.767225,0.705513,0.177131,...,0.028385,0.060367,0.135101,0.155464,0.121900,0.097025,0.059115,0.023723,-0.009320,-0.001518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,-0.045691,-0.012598,0.025667,0.052823,0.097025,0.058563,0.086607,0.103381,0.107921,0.030348,...,-0.041217,0.068082,0.194290,0.309536,0.782268,1.000000,0.683729,0.268356,0.135452,0.114957
554,-0.037065,-0.017195,0.002583,0.029776,0.059115,0.050686,0.053207,0.063209,0.062769,0.031297,...,-0.009569,0.051610,0.131266,0.223666,0.578782,0.683729,1.000000,0.624946,0.441509,0.286747
555,-0.027702,-0.021747,-0.016028,-0.021659,0.023723,0.014708,0.053820,0.064009,0.067066,0.034825,...,0.118118,0.140593,0.134387,0.154893,0.201428,0.268356,0.624946,1.000000,0.900797,0.801243
556,-0.006768,-0.004366,-0.020960,-0.044240,-0.009320,-0.014925,0.020486,0.029279,0.029749,0.019052,...,0.163637,0.166037,0.119591,0.105379,0.051543,0.135452,0.441509,0.900797,1.000000,0.872146


In [24]:
myarray = np.array(corr)

In [25]:
dm = mdtraj.compute_contacts(ref.atom_slice(top.select('protein and not (resname CAL)')))
dm =  mdtraj.geometry.squareform(dm[0], dm[1])[0]

In [26]:
table_dist = dm 
table_corr = corr

In [27]:
print('table_corr.shape:',table_corr.shape)

table_corr.shape: (558, 558)


In [28]:
table_dist.shape

(558, 558)

In [29]:
table_corr.shape

(558, 558)

In [30]:
if table_corr.shape != table_dist.shape:
    print('table_corr.shape:',table_corr.shape)
    print('table_dist.shape:',table_corr.shape)
    raise ValueError('table_corr and table_dist should have the same shape')
    
fila=0
distance=6.0

valors=dict()

valors={'nodes':[],'edges':[],'weights':[], 'residues':[]}

while fila<(len(table_dist)):
    
    columna=0
    
    valors["nodes"].append(fila)
    valors["residues"].append(str(fila+1))
    while columna<len(table_dist[fila]):
        if (float(table_dist[fila][columna])<float(distance) and (fila < columna)):
            valors["edges"].append([fila,columna])
            valors["weights"].append(math.fabs(-math.log10(math.fabs(float(table_corr[fila][columna])))))
        #			print fila,columna,table_corr[fila][columna],math.fabs(-math.log10(math.fabs(float(table_corr[fila][columna]))))
        columna=columna+1
    fila=fila+1


In [31]:
g = igraph.Graph()

# assigno dades al graph

g.add_vertices(valors["nodes"])
g.add_edges(valors["edges"])
g.es["weight"]=valors["weights"]

g.vs["residues"]=valors["residues"]

In [32]:
def computeShortestPaths(graph,nodes,weights,output_type):
    shortest_paths=[]

    for node_idx_origin in tqdm(nodes):
    
        node_idx_destination=node_idx_origin+2
    
        while (node_idx_destination < len(nodes)):
            shortest_paths.append(graph.get_shortest_paths(node_idx_origin,node_idx_destination, weights, "OUT",output_type))
            node_idx_destination=node_idx_destination+1

    return(shortest_paths)


shortest_paths=computeShortestPaths(g,valors["nodes"],valors["weights"],"vpath")

sp_graph = igraph.Graph()

sp_graph.add_vertices(valors["nodes"])

edges_already_added = [[0 for col in range(len(valors["nodes"]))] for row in range(len(valors["nodes"]))]


list_edges_shortest_paths=[]
list_edges=[]



100%|█████████████████████████████████████████| 558/558 [05:28<00:00,  1.70it/s]


In [33]:

for path_vertices in shortest_paths:
    path_vertices=path_vertices[0]
    
    for i in range(len(path_vertices)-1):
        num_edges = edges_already_added[path_vertices[i]][path_vertices[i+1]]
        if( num_edges == 0):
            list_edges_shortest_paths.append([path_vertices[i],path_vertices[i+1]])
        edges_already_added[path_vertices[i]][path_vertices[i+1]] = num_edges + 1
        edges_already_added[path_vertices[i+1]][path_vertices[i]] = num_edges + 1




sp_graph.add_edges(list_edges_shortest_paths)


In [47]:
list_weights_shortest_paths=[]
list_weights_shortest_paths_unweighted=[]
labels_weighted_nodes=[""] * len(valors["nodes"])
vertex_sizes=["0"] * len(valors["nodes"])

In [48]:
def GetMaximumValueMatrix(matrix):
    maximum_value=0
    
    for list in matrix:
        for value in list:
            if (value > maximum_value):
                maximum_value=value
    return(maximum_value)

maximum_weight=float(GetMaximumValueMatrix(edges_already_added))


#print "****************************************"
#print "Maxim weight trobat per tots els edges que composen matriu: "+ str(maximum_weight)


for edge in list_edges_shortest_paths:
    reps = edges_already_added[edge[0]][edge[1]]/maximum_weight
    list_weights_shortest_paths_unweighted.append(edges_already_added[edge[0]][edge[1]])
    if(reps > 0.3):
        list_weights_shortest_paths.append(reps*10)
        labels_weighted_nodes[edge[0]] = valors["residues"][edge[0]]
        labels_weighted_nodes[edge[1]] = valors["residues"][edge[1]]
        vertex_sizes[edge[0]] = "8"
        vertex_sizes[edge[1]] = "8"
    else:
        list_weights_shortest_paths.append(0)


In [49]:
average_path_length=g.average_path_length()

print("****************************************")
print("Calculo el average path length:")
print(average_path_length)

#Calculo ara la contribucio de cada residue k a la network recalculant de nou el CPL despres de treure el node k de la network
betweenness=g.betweenness(weights=g.es["weight"])

print("****************************************")
print("Calculo ara el betweeness de cada vertex del network")
print(betweenness)

****************************************
Calculo el average path length:
1.005257298765146
****************************************
Calculo ara el betweeness de cada vertex del network
[146.0, 107.0, 365.0, 1040.0, 1530.0, 0.0, 502.0, 1234.0, 707.0, 0.0, 144.0, 133.0, 0.0, 49.0, 6.0, 3.0, 367.0, 94.0, 416.0, 89.0, 1416.0, 153.0, 248.0, 10.0, 1017.0, 929.0, 0.0, 207.0, 1.0, 11.0, 299.0, 291.0, 296.0, 0.0, 0.0, 65.0, 1530.0, 69.0, 723.0, 453.0, 31.0, 39.0, 621.0, 968.0, 1409.0, 336.0, 689.0, 0.0, 1794.0, 2772.0, 1.0, 52.0, 60.0, 9.0, 888.0, 419.0, 24.0, 66.0, 1540.0, 520.0, 494.0, 14.0, 31.0, 425.0, 11.0, 501.0, 1447.0, 396.0, 9.0, 8.0, 38.0, 3.0, 0.0, 0.0, 103.0, 405.0, 389.0, 0.0, 10.0, 19.0, 305.0, 59.0, 6.0, 224.0, 0.0, 0.0, 21.0, 167.0, 2.0, 386.0, 6.0, 2134.0, 1706.0, 503.0, 339.0, 189.0, 82.0, 75.0, 94.0, 340.0, 1.0, 4.0, 91.0, 51.0, 76.0, 82.0, 0.0, 335.0, 206.0, 422.0, 873.0, 8.0, 0.0, 19.0, 0.0, 58.0, 2.0, 43.0, 100.0, 177.0, 155.0, 337.0, 756.0, 128.0, 198.0, 892.0, 895.0, 712

In [50]:
if (community_method == "edge_betweness"):
    VertexDendrogram = g.community_edge_betweenness(None, False, "weight")

if (community_method == "walktrap"):
    VertexDendrogram=g.community_walktrap( "weight", 4)

if (community_method == "fastgreedy"):
    VertexDendrogram=g.community_fastgreedy( "weight")

#vertexClustering = g.community_infomap("weight")
community_graph = VertexDendrogram.as_clustering(VertexDendrogram.optimal_count).cluster_graph(None,"sum")

In [51]:
def writeFile(filename,text):
    file=open(filename,"w+")
    
    file.write(text)

    file.close()

In [52]:
def get_color(index):
    colors=[]

    mediumseagreen=[60,179,113]
    dark_orange=[255,140,0]
    slate_blue=[106,90,205]
    crimson=[220,20,60]
    medium_orchid=[186,85,211]
    gold=[255,215,0]
    medium_turquoise=[72,209,204]
    deep_sky_blue=[0,191,255]
    teal=[0,128,128]
    deep_pink=[255,20,147]
    midnight_blue=[25,25,112]
    dark_sea_green=[143,188,143]
    orange_red=[255,69,0]
    green_yellow=[173,255,47]
    dark_blue=[0,0,139]
    olive=[128,128,0]
    plum=[221,160,221]
    slate_gray=[112,128,144]
    salmon=[250,128,114]
    purple=[128,0,128]
    black=[0,0,0]
    dark_red=[139,0,0]
    dark_violet=[148,0,211]
    khaki=[240,230,140]
    hot_pink=[255,105,180]
    aqua=[0,255,255]
    silver=[192,192,192]
    limegreen=[50,205,50]
    magenta=[255,0,255]
    lightslategray=[119,136,153]
    khaki=[240,230,140]
        
    colors.append(mediumseagreen)
    colors.append(dark_orange)
    colors.append(slate_blue)
    colors.append(crimson)
    colors.append(medium_orchid)
    colors.append(gold)
    colors.append(medium_turquoise)
    colors.append(plum)
    colors.append(teal)
    colors.append(deep_pink)
    colors.append(midnight_blue)
    colors.append(green_yellow)
    colors.append(deep_sky_blue)
    colors.append(purple)
    colors.append(dark_blue)
    colors.append(olive)
    colors.append(orange_red)
    colors.append(slate_gray)
    colors.append(salmon)
    colors.append(dark_sea_green)
    colors.append(black)
    colors.append(dark_red)
    colors.append(dark_violet)
    colors.append(khaki)
    colors.append(hot_pink)
    colors.append(aqua)
    colors.append(silver)
    colors.append(limegreen)
    colors.append(magenta)
    colors.append(lightslategray)
    colors.append(khaki)

    if (index<0 or index>len(colors)-1):
        return random_color()
    else:
        return colors[index]

def random_color():
    #	a = float(random.randrange(0,255))
    #	b = float(random.randrange(0,255))
    #	c = float(random.randrange(0,255))


    #	a2=a/255
    #	b2=b/255
    #	c2=c/255

    a2=int(int(random.randrange(0,255)))
    b2=int(int(random.randrange(0,255)))
    c2=int(int(random.randrange(0,255)))

    color=[a2,b2,c2]
    
    return(color)


def multiplyListValues(list,multiplicator):
    new_list=[]
    for value in list:
        new_list.append(value*multiplicator)

    return(new_list)

def findMaxMinValue(list):
    maximum_value=0
    minimum_value=0
    for value in list:
        if value > maximum_value:
            maximum_value=value
        if value < minimum_value:
            minimum_value=value
    return(maximum_value,minimum_value)



In [53]:
community_vertices_weights = []
community_vertices_names = []

list_community_vertices_names=[]


for graph in VertexDendrogram.as_clustering(VertexDendrogram.optimal_count).subgraphs():
    #print graph.vs["residues"]
    community_vertices_weights.append(len(graph.vs["residues"]))
    community_vertices_names.append(' '.join(graph.vs["residues"]))
    list_community_vertices_names.append(graph.vs["residues"])


community_graph.vs["size"] = community_vertices_weights
#community_graph.vs["label"] = community_vertices_names


In [54]:
i = 0

community_vertices_colors=[]
community_vertices_node_names=[]

text="hide everything\n"
text=text+"show ribbon\n"
color_python=[]

while (i< len(community_vertices_names)):
    name_replaced = re.sub('\s+', '+', community_vertices_names[i])
    text=text+"select node "+str(i)+", resi "+name_replaced+"\n"
    color_pymol=get_color(i)
    node_label=""
    string_color="rgb("
    el_count = 0
    for el in color_pymol:
        if (el_count < 2):
            string_color=string_color+str(el)+","
        else:
            string_color=string_color+str(el)
        el_count = el_count + 1
    string_color=string_color+")"
    community_vertices_colors.append(string_color)
    text=text+"set_color c"+str(i)+"="+str(color_pymol)+"\n"
    text=text+"color c"+str(i)+", (node_"+str(i)+")\n"
    text=text+"label first resn ala in node_"+str(i)+", "+str(i)+"\n"
    node_label=str(i)
    community_vertices_node_names.append(node_label)
    
    i=i+1

In [55]:
#print community_vertices_colors

writeFile("pymol_community_analysis_"+figure_name,text)

# Assigno colors al graph

community_graph.vs["color"] = community_vertices_colors
community_graph.vs["label"] = community_vertices_node_names
community_graph.vs["label_dist"]= 0
#community_graph.vs["label_size"]= 20
community_graph.vs["label_color"]= "white"
#community_graph.vs["bbox"] = (1000, 1000)
#community_graph.vs["margin"] = 200



visual_style = {}
visual_style["vertex_size"] = community_vertices_weights
visual_style["edge_width"] = multiplyListValues(community_graph.es["weight"],10)

#Graph community


igraph.plot(community_graph,figure_name+".png",**visual_style)

#Graph de shortest path

visual_style = {}
visual_style["vertex_size"] = vertex_sizes
visual_style["edge_width"] = list_weights_shortest_paths
visual_style["vertex_label"] = labels_weighted_nodes
visual_style["vertex_label_size"] = "12"
visual_style["vertex_label_dist"] = "2"
#visual_style["vertex_label_angle"] = "15"



colors_weighted_nodes=["rgb(255,255,255)"] * len(valors["nodes"])

# Assigno el mateix color als nodes de graph de shortest path que als de community

i=0
for label in labels_weighted_nodes:
    if (label != ""):
        node=0
        for list in list_community_vertices_names:
            for element in list:
                if (label == element):
                    colors_weighted_nodes[i] =community_vertices_colors[node]
            node=node+1
    i=i+1



sp_graph.vs["color"] = colors_weighted_nodes


igraph.plot(sp_graph,figure_name+"_shortest_paths.png",**visual_style)


text=""
text=text+"hide everything\n"
text=text+"show cartoon\n"
text=text+"set stick_color, black\n"
text=text+"bg_color white\n"

text_output=""
text_output=text_output+" Residue1 - Residue2 - Weight Path\n"
group=""

for index in range(len(list_weights_shortest_paths)):
    edge = list_edges_shortest_paths[index]
    weight = list_weights_shortest_paths[index]

    if weight > 0 :
        #print str(edge[0]+1)+" "+str(edge[1]+1)+" "+str(weight)

        text_output=text_output+str(edge[0]+1)+" "+str(edge[1]+1)+" "+str(weight)+"\n"

        text=text+"create test"+str(index)+", name ca in resi "+str(edge[0]+1)+"+"+str(edge[1]+1)+"\n"
        text=text+"show spheres, test"+str(index)+"\n"
        text=text+"set sphere_scale,"+str(0.1+weight/10)+", test"+str(index)+"\n"
        text=text+"bond name ca in resi "+str(edge[0]+1)+", name ca in resi "+str(edge[1]+1)+"\n"
        text=text+"set stick_radius,"+str(weight/10)+", test"+str(index)+"\n"
        text=text+"show sticks, test"+str(index)+"\n"
        group=group+"test"+str(index)+" "

print("****************************************")

text=text+"group PATH, "+group

writeFile("pymol_shortest_path_"+figure_name,text)
writeFile("output_shortest_path_"+figure_name+".out",text_output)

****************************************
